In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 521.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) 

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Text'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Text'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Text'].tolist()
y_test=test['Sentiment'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['angry','disgust','fear','happy','sad','surpise']

In [12]:
model_name = 'xlm-roberta-base'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 102


Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 49
	99percentile : 96


In [14]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 1964s 5s/step - loss: 1.7900 - accuracy: 0.2256 - val_loss: 1.7005 - val_accuracy: 0.3900
Epoch 2/20
427/427 [==============================] - 1884s 4s/step - loss: 1.6561 - accuracy: 0.3817 - val_loss: 1.4684 - val_accuracy: 0.4727
Epoch 3/20
427/427 [==============================] - 1866s 4s/step - loss: 1.5436 - accuracy: 0.4456 - val_loss: 1.4186 - val_accuracy: 0.4680
Epoch 4/20
427/427 [==============================] - 1867s 4s/step - loss: 1.4824 - accuracy: 0.4738 - val_loss: 1.4450 - val_accuracy: 0.4727
Epoch 5/20
427/427 [==============================] - 1888s 4s/step - loss: 1.3944 - accuracy: 0.5043 - val_loss: 1.3607 - val_accuracy: 0.4945
Epoch 6/20
427/427 [==============================] - 1871s 4s/step - loss: 1.2829 - accuracy: 0.5523 - val_loss: 1.5107 - val_accuracy: 0.4509
Epoch 7/20
427/427 [==============================] - 1882s 4s/step - los

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       angry       0.56      0.61      0.58       137
     disgust       0.24      0.26      0.25        72
        fear       0.21      0.20      0.21        30
       happy       0.74      0.74      0.74       195
         sad       0.43      0.45      0.44       132
     surpise       0.38      0.24      0.29        75

    accuracy                           0.52       641
   macro avg       0.43      0.42      0.42       641
weighted avg       0.51      0.52      0.51       641



array([[ 84,  20,   5,   5,  17,   6],
       [ 21,  19,   0,  11,  15,   6],
       [  2,   3,   6,   4,  11,   4],
       [  6,  10,   3, 145,  24,   7],
       [ 22,  17,   9,  17,  60,   7],
       [ 16,  11,   5,  13,  12,  18]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [20]:
y_pred = predictor.predict(x_test)
print(y_pred)

['angry', 'disgust', 'angry', 'disgust', 'sad', 'surpise', 'happy', 'fear', 'happy', 'surpise', 'disgust', 'angry', 'happy', 'happy', 'angry', 'disgust', 'happy', 'surpise', 'happy', 'happy', 'happy', 'angry', 'disgust', 'disgust', 'angry', 'happy', 'happy', 'sad', 'angry', 'happy', 'happy', 'disgust', 'sad', 'happy', 'sad', 'angry', 'sad', 'angry', 'happy', 'happy', 'angry', 'surpise', 'happy', 'happy', 'angry', 'sad', 'sad', 'sad', 'surpise', 'angry', 'angry', 'angry', 'surpise', 'happy', 'fear', 'angry', 'sad', 'sad', 'disgust', 'disgust', 'angry', 'sad', 'sad', 'happy', 'disgust', 'surpise', 'fear', 'sad', 'happy', 'sad', 'happy', 'sad', 'happy', 'happy', 'sad', 'angry', 'sad', 'angry', 'sad', 'disgust', 'disgust', 'angry', 'happy', 'angry', 'angry', 'happy', 'happy', 'happy', 'disgust', 'happy', 'sad', 'sad', 'happy', 'happy', 'sad', 'angry', 'fear', 'angry', 'angry', 'angry', 'angry', 'angry', 'sad', 'angry', 'surpise', 'surpise', 'happy', 'happy', 'sad', 'sad', 'happy', 'surpise

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.58      0.59      0.58       144
     disgust       0.18      0.18      0.18        66
        fear       0.09      0.09      0.09        32
       happy       0.74      0.80      0.77       187
         sad       0.48      0.44      0.46       148
     surpise       0.24      0.21      0.22        63

    accuracy                           0.51       640
   macro avg       0.38      0.39      0.38       640
weighted avg       0.50      0.51      0.51       640

